# Escaleras y serpientes (tarea 2 RL)


<center>
    <img src="img_hw2/escalera.png" alt="centered image" width=300/>
</center>

Se tiene el siguiente juego de escalera, hay serpientes que hacen descender y escaleras que hacen ascender. Los puntos rojos y verdes son casillas en donde se va a terminar el juego, si cae en la casilla roja se pierde la partida, si cae en la casilla azul se gana la partida.

La idea es utilizar diferentes métodos de aprendizaje por refuerzo para ver cómo se comporta con el juego.

### Reglas del juego

* El jugador decide si avanzar o retroceder la cantidad de casillas que indique el dado **Antes** de lanzarlo
* Si se llega a uno de los bordes de la escalera (1 o 100), el jugador "rebotará" en las casillas y retrocederá o avanzará (dependiendo el caso) el número de casillas restantes.

## Modelado como MDP

Este juego posee las siguientes características:

* Estados -> 100. Estos corresponden a las posiciones posibles del tablero. Cuando el agente llega al inicio de una escalera o en la cabeza de una serpiente, el estado al que va a seguir el agente va a ser desde el otro lado de la escalera/serpiente. 
* Recompensas -> 1. si el jugador llega a un punto azul, -1 si el jugador llega a un punto rojo. 0 de lo contrario.
* Acciones -> 2. El agente solo se puede mover en dos direcciones: adelante (aumentando casillas) o atrás (retrocediendo).

Lo primero que se debe hacer es importar el módulo realizado para la solución de la tarea y crear el tablero. Siempre va a ser un tablero de 100 casillas, pero es posible cambiar tanto las esca

In [1]:
from game_hw2 import Game_hw2

snakes = [(73,1), (46,5), (55,7), (48,9), (52,11), (59,17), (83,19), (44,22), (95,24), (98,28), (69,33), (64,36), (92,51)]
stairs = [(8,26), (21,82), (43,77), (50,91), (54,93), (62,96), (66,87), (80,100)]
goals =[80, 100] 
fails =[23, 37, 45, 67, 89] 
gw = Game_hw2(goals = goals, fails = fails, snakes = snakes, stairs = stairs)

Probar función de pasos para estados resultantes:

In [8]:
start_point = 5
valor_dado = 5
next_state, reward, _, done, dice = gw.step(start_point, 'Ad', dice = valor_dado) 

print(next_state, reward, _, done, dice)

10 0 Ad False 5


Probar función de pasos para acciones del agente.

Estructura de retorno:
* Estado siguiente, recompensa estado, Accion tomada, Estado terminal, valor del dado lanzado

In [9]:
state_prueba=98
action_prueba='Ad'
for i in range(6*3):
    print(gw.step(state_prueba, action_prueba))

(99, 0, 'Ad', False, 1)
(99, 0, 'Ad', False, 1)
(97, 0, 'Ad', False, 5)
(99, 0, 'Ad', False, 3)
(99, 0, 'Ad', False, 3)
(97, 0, 'Ad', False, 5)
(100, 1, 'Ad', True, 2)
(100, 1, 'Ad', True, 2)
(28, 0, 'Ad', False, 4)
(99, 0, 'Ad', False, 3)
(99, 0, 'Ad', False, 1)
(97, 0, 'Ad', False, 5)
(99, 0, 'Ad', False, 1)
(97, 0, 'Ad', False, 5)
(97, 0, 'Ad', False, 5)
(100, 1, 'Ad', True, 2)
(100, 1, 'Ad', True, 2)
(96, 0, 'Ad', False, 6)


## Algoritmo Value iteration

In [2]:
def value_iteration(gw, gamma):
    # for que recorre todos los estados
    for state in range(1,101):
        q = {'Ad': 0,
             'At': 0}
        # for que ingresa a todas las posibles acciones del estado (Ad, At)
        for posible_action in gw.probabilities[state]:
            # vector que tiene los 6 posibles resultados de la accion actual (1-6, val dados)
            current_action = gw.probabilities[state][posible_action]
            total_sum_action =[]
            
            # for que revisa cada posible resultado del dado
            dice = 1
            for action in current_action:
                # Da un paso hacia la direccion Posible_action con el valor del dado dice
                new_state, reward, _, done, dice = gw.step(state, posible_action, dice = dice)
                if (state in gw.goals): total_sum_action = [gw.goal_rew]
                elif state in gw.fails: total_sum_action = [gw.fail_rew]
                else:
                    total_sum_action.append(reward+action[0]*(gamma*gw.state_values[new_state]))
                   
                dice+=1
            # Suma todos los valores obtenidos en cada estado y le asigna a q[stado_actual] el valor maximo
            q[posible_action] = sum(total_sum_action)
            # Coloca en la politica el valor maximo de la accion (si fue Ad o At)
            max_value = max(q, key=q.get)
            
            # Asigna el valor maximo de la politica
            gw.policy[state] = max(q, key=q.get)
            gw.state_values[state] = q[max_value]
        


Prueba del algoritmo *Value iteration*

In [3]:
gamma = 0.9
H = 90
for i in range(H):
    value_iteration(gw,0.9)


In [4]:
for state in range(1,101):
    print(f'V({state}) = {gw.state_values[state]:.3f}')


V(1) = -3.470
V(2) = -3.017
V(3) = -2.966
V(4) = -2.915
V(5) = -3.353
V(6) = -3.411
V(7) = -3.470
V(8) = -3.470
V(9) = -3.523
V(10) = -3.546
V(11) = -3.523
V(12) = -3.503
V(13) = -3.486
V(14) = -3.471
V(15) = -2.827
V(16) = -2.827
V(17) = -3.372
V(18) = -3.372
V(19) = -3.372
V(20) = -3.372
V(21) = -3.471
V(22) = -2.827
V(23) = -1.000
V(24) = -3.372
V(25) = -3.372
V(26) = -3.372
V(27) = -3.372
V(28) = -4.097
V(29) = -4.286
V(30) = -3.881
V(31) = -3.957
V(32) = -4.045
V(33) = -4.146
V(34) = -4.262
V(35) = -4.286
V(36) = -4.202
V(37) = -1.000
V(38) = -3.001
V(39) = -3.522
V(40) = -3.500
V(41) = -3.492
V(42) = -3.496
V(43) = -4.113
V(44) = -3.027
V(45) = -1.000
V(46) = -3.442
V(47) = -3.437
V(48) = -3.253
V(49) = -3.281
V(50) = -3.492
V(51) = -3.446
V(52) = -3.410
V(53) = -3.405
V(54) = -3.437
V(55) = -3.253
V(56) = -3.143
V(57) = -3.143
V(58) = -3.281
V(59) = -3.181
V(60) = -2.804
V(61) = -3.352
V(62) = -3.455
V(63) = -3.143
V(64) = -3.058
V(65) = -2.707
V(66) = -2.695
V(67) = -1.000
V(68

In [5]:
print(gw.policy)

{1: 'Ad', 2: 'At', 3: 'At', 4: 'At', 5: 'At', 6: 'At', 7: 'At', 8: 'At', 9: 'At', 10: 'Ad', 11: 'Ad', 12: 'Ad', 13: 'Ad', 14: 'Ad', 15: 'Ad', 16: 'Ad', 17: 'Ad', 18: 'Ad', 19: 'Ad', 20: 'Ad', 21: 'At', 22: 'At', 23: 'Ad', 24: 'At', 25: 'At', 26: 'At', 27: 'At', 28: 'At', 29: 'Ad', 30: 'At', 31: 'At', 32: 'At', 33: 'At', 34: 'At', 35: 'At', 36: 'Ad', 37: 'Ad', 38: 'Ad', 39: 'Ad', 40: 'Ad', 41: 'Ad', 42: 'Ad', 43: 'Ad', 44: 'At', 45: 'Ad', 46: 'Ad', 47: 'Ad', 48: 'Ad', 49: 'Ad', 50: 'Ad', 51: 'Ad', 52: 'Ad', 53: 'Ad', 54: 'At', 55: 'At', 56: 'Ad', 57: 'Ad', 58: 'Ad', 59: 'Ad', 60: 'Ad', 61: 'Ad', 62: 'Ad', 63: 'At', 64: 'Ad', 65: 'Ad', 66: 'Ad', 67: 'Ad', 68: 'Ad', 69: 'Ad', 70: 'Ad', 71: 'Ad', 72: 'Ad', 73: 'Ad', 74: 'Ad', 75: 'Ad', 76: 'Ad', 77: 'Ad', 78: 'Ad', 79: 'Ad', 80: 'Ad', 81: 'At', 82: 'At', 83: 'At', 84: 'At', 85: 'At', 86: 'At', 87: 'At', 88: 'At', 89: 'Ad', 90: 'At', 91: 'At', 92: 'At', 93: 'Ad', 94: 'Ad', 95: 'Ad', 96: 'Ad', 97: 'Ad', 98: 'Ad', 99: 'Ad', 100: 'Ad'}


In [17]:
"""
Pruebas para hacer lo de rebote
a = 95
pasos = []
for i in range(a,101):
    for dice in range(1,7):
        if i+dice>=101:
            print("casilla: "+ str(i)+" _ "+ "dado: "+str(dice))
            print("estoy en ",i+dice, " voy a moverme a", 100 - (i+dice -100)) 
            print("-"*20)

a = 6
for i in range(a,0,-1):
    for dice in range(1,7):
        if i-dice<=0:
            print('casilla: '+str(i)+" _ "+'dado: '+str(dice))
            print('Estoy en ', i-dice, 'voy a moverme a', abs(i-dice)+2)
            print("-"*20)
"""
print("")